In [5]:
from pydantic import BaseModel, Field
import json
from simpleaichat import AIChat
from simpleaichat.utils import fd, remove_a_key

In [6]:
class answer_question(BaseModel):
    """Returns an answer to a question the user asked."""

    answer: int = Field(description="Answer to the user's question.")
    ones_name: str = Field(description="Name of the ones digit of the answer.")

In [7]:
print(json.dumps(answer_question.model_json_schema(), indent=2))

{
  "description": "Returns an answer to a question the user asked.",
  "properties": {
    "answer": {
      "description": "Answer to the user's question.",
      "title": "Answer",
      "type": "integer"
    },
    "ones_name": {
      "description": "Name of the ones digit of the answer.",
      "title": "Ones Name",
      "type": "string"
    }
  },
  "required": [
    "answer",
    "ones_name"
  ],
  "title": "answer_question",
  "type": "object"
}


Demo output for what gets sent to the model:


In [14]:
schema_dict = answer_question.model_json_schema()
remove_a_key(schema_dict, "title")

data = {
    "tools": [
        {
            "name": answer_question.__name__,
            "description": answer_question.__doc__,
            "parameters": schema_dict,
        }
    ],
    "tool_choice": {"type": "function", "function": {"name": answer_question.__name__}},
}

print(json.dumps(data, indent=2))

{
  "tools": [
    {
      "name": "answer_question",
      "description": "Returns an answer to a question the user asked.",
      "parameters": {
        "description": "Returns an answer to a question the user asked.",
        "properties": {
          "answer": {
            "description": "Answer to the user's question.",
            "type": "integer"
          },
          "ones_name": {
            "description": "Name of the ones digit of the answer.",
            "type": "string"
          }
        },
        "required": [
          "answer",
          "ones_name"
        ],
        "type": "object"
      }
    }
  ],
  "tool_choice": {
    "type": "function",
    "function": {
      "name": "answer_question"
    }
  }
}


In [ ]:
data

In [4]:
ai = AIChat(
    console=False,
    save_messages=False,
    model="gpt-3.5-turbo",
    params={"temperature": 0.0},
)

In [5]:
response_structured = ai(
    "How many miles is it from San Francisco to Los Angeles?",
    output_schema=answer_question,
)

print(json.dumps(response_structured, indent=2))

{
  "answer": 382,
  "ones_name": "two"
}


Swapping the fields to show that order matters.


In [6]:
class answer_question_r(BaseModel):
    """Returns an answer to a question the user asked."""

    ones_name: str = Field(description="Name of the ones digit of the answer.")
    answer: int = Field(description="Answer to the user's question.")


response_structured = ai(
    "How many miles is it from San Francisco to Los Angeles?",
    output_schema=answer_question_r,
)

print(json.dumps(response_structured, indent=2))

{
  "ones_name": "miles",
  "answer": 382
}


Code question


In [7]:
class answer_code_question(BaseModel):
    """Returns an answer to a coding question the user asked."""

    code: str = Field(description="Code the user requested, without code comments.")


response_structured = ai(
    "Write a Python function to detect whether a string is a palindrome, as efficiently as possible.",
    output_schema=answer_code_question,
)

print(json.dumps(response_structured, indent=2))

{
  "code": "def is_palindrome(s):\n    return s == s[::-1]"
}


In [8]:
class answer_code_question(BaseModel):
    """Returns an answer to a coding question the user asked."""

    code: str = fd("Code the user requested, without code comments.")
    optimized_code: str = fd(
        "Algorithmically optimized code from the previous response."
    )


response_structured = ai(
    "Write a Python function to detect whether a string is a palindrome, as efficiently as possible.",
    output_schema=answer_code_question,
)

print(json.dumps(response_structured, indent=2))

{
  "code": "def is_palindrome(s):\n    return s == s[::-1]",
  "optimized_code": "def is_palindrome(s):\n    left = 0\n    right = len(s) - 1\n    while left < right:\n        if s[left] != s[right]:\n            return False\n        left += 1\n        right -= 1\n    return True"
}
